In [3]:
import pandas as pd
import numpy as np



from session_config import  collect_survey_data, palette
from userdisplay import table_css_styles, highlight_max, format_kwargs
from reports import make_report_objects, reports_and_forecast

from reports import admin_report, features_present, histograms_standard
from reports import ecdf_plots_standard, scatter_plot_standard
from reports import labels_for_display, make_standard_report, make_report_objects


from functools import wraps

import geopandas as gpd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from IPython.display import display, Markdown
from myst_nb import glue


from shapely.geometry import Point

beaches = pd.read_csv('data/end_process/beaches.csv')
lat_lon = beaches[['slug', 'latitude', 'longitude']].set_index('slug')

def display_selected_columns(data, columns: [] = None, index_column: str = None):
    
    if columns != None:
        data = data[columns]
    if index_column != None:
        data.set_index(index_column, inplace=True, drop=True)
    return data 


def map_markers(df, lat_lon: pd.DataFrame = lat_lon):
    """Map the markers"""
    nsamples = df.groupby('location', observed=True)['sample_id'].nunique()
    qty_location = df.groupby('location', observed=True)['quantity'].sum()
    rate_location = df.groupby('location', observed=True)['pcs/m'].mean().round(2)
    last_sample = df.groupby('location', observed=True)['date'].max()
    df = pd.concat([nsamples, qty_location, rate_location, last_sample], axis=1)
    df = df.merge(lat_lon, left_index=True, right_index=True)
    df['location'] = df.index
    max_lat, min_lat = df['latitude'].max(), df['latitude'].min()
    max_lon, min_lon = df['longitude'].max(), df['longitude'].min()
    td = df.to_dict(orient='records')
    geometry = [Point(loc['longitude'], loc['latitude']) for loc in td]
    geo_frame = gpd.GeoDataFrame(td, geometry=geometry, crs="EPSG:4326")
    
    return geo_frame , {'max_lat': max_lat, 'min_lat': min_lat, 'max_lon': max_lon, 'min_lon': min_lon}

def scatter_plot_display(report_results, standard_report):

    points = []
    dates = "" 
    
    if report_results['prior_report'] != 'No prior':
        pp = (report_results['prior_report'].sample_results, standard_report['prior-labels'], palette['prior'])
        points.append(pp)
        dates += standard_report['prior-labels'][:4]
        dates += ' - ' 
    if report_results['this_report'] != 'No likelihood':
        lp = (report_results['this_report'].sample_results, standard_report['likelihood-labels'], palette['likelihood'])
        points.append(lp)
        dates += standard_report['likelihood-labels'][-4:]
    
    title = f'All samples {canton}: {dates}'
    
    

    return scatter_plot_standard(points), title


def add_format_decorator(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        format_kwargs = kwargs.pop('format_kwargs', None)
        result = func(*args, **kwargs)
        if format_kwargs and hasattr(result, 'format'):
            result = result.format(**format_kwargs)
        return result
    return wrapper

@add_format_decorator
def apply_table_styles(table, table_style: [] = None):
    index = [x for x in table.index if x not in ['start', 'end']]
    table = table.loc[index]
        
    if table_style != None:
        return table.style.set_table_styles(table_style)
    else:
        return table.style.set_table_styles(table_css_styles)

# palette = {'prior':'goldenrod', 'likelihood':'dodgerblue'}

datax = collect_survey_data()

# from use_cases example
ooi = ['G10',  'Gcaps', 'G30', 'G31', 'G33', 'G34', 'G35', 'G8', 'G7', 'G6', 'G5', 'G4', 'G37', 'G2', 'G27', 'G25']
# more refined search
tobo_snacks = ['G27', 'G30', 'G35']
# unidentified, plastic, different uses
udi = ['Gfrags', 'Gfoams']
# industrial
indus = ['G89', 'G67', 'G112', 'G93' , 'G66' 'G74', 'G72', 'Gfrags', 'Gfoams']

# features
land_covers = ['buildings', 'forest', 'undefined', 'public services', 'streets']

In [5]:
o_dates = {'start':'2020-01-01', 'end':'2021-12-31'}
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}

# 'Neuchâtel', 'Zürich', 


canton = 'Bern'
this_feature_type = 'l'

d= datax.reset_index(drop=True)


# make complete report
params_l = {'canton':canton, 'date_range':o_dates, 'feature_type': this_feature_type}
params_p = {'canton':canton, 'date_range':prior_dates, 'feature_type':this_feature_type}

# set the parameters for the weighted prior
# exclude records in the likelihood, set date range and feature type
# make the land-use-inventory, exclude any likelihood values
lu_catalogue = d[(d.canton != canton)&(d['date'] <= o_dates['end'])].copy()
catalog_surveys, catalog_features = make_report_objects(lu_catalogue)
combined_land_use_catalog = catalog_features.df_cat.copy()
# lake_land_use_catalog['feature_type'] = 'l' 


all_data_of_interest = d[(d.canton == canton) & (d['date'] >= prior_dates['start']) & (d['date'] <= o_dates['end'])].copy()
all_data_of_interest = all_data_of_interest.copy()
all_data_of_interest.reset_index(inplace=True, drop=True)

all_data_combined, all_land_use_combined = make_report_objects(all_data_of_interest)

args = {
    'likelihood': {'canton':canton, 'date_range':o_dates},
    'prior' : {'canton':canton, 'date_range':prior_dates},
    'data' : all_data_of_interest,
    'land-use-inventory' : combined_land_use_catalog
}


combined_results = reports_and_forecast(args['likelihood'], args['prior'], ldata=args['data'])
standard_combined = make_standard_report(combined_results, args)


# lakes
data_of_interest = all_data_of_interest[(all_data_of_interest.feature_type == this_feature_type)].copy()
data_of_interest = data_of_interest.copy()
data_of_interest.reset_index(inplace=True)

lake_combined, lake_land_use_combined = make_report_objects(data_of_interest)

lake_catalogue = lu_catalogue[lu_catalogue.feature_type == 'l'].copy()
catalog_surveys, catalog_features = make_report_objects(lake_catalogue)

lake_land_use_catalog = catalog_features.df_cat.copy()
lake_land_use_catalog['feature_type'] = 'l' 

args = {
    'likelihood': params_l,
    'prior' : params_p,
    'data' : data_of_interest,
    'land-use-inventory' : lake_land_use_catalog
}

lake_results = reports_and_forecast(args['likelihood' ], args['prior'], ldata=args['data'])
standard_lake = make_standard_report(lake_results, args)

# rivers
data_of_interest = all_data_of_interest[(all_data_of_interest.canton == canton)&(all_data_of_interest.feature_type == 'r')].copy()
data_of_interest = data_of_interest.copy()
data_of_interest.reset_index(inplace=True)

river_combined, river_land_use_combined = make_report_objects(data_of_interest)


params_l.update({'feature_type': 'r'})
params_p.update({'feature_type': 'r'})

river_catalogue = lu_catalogue[lu_catalogue.feature_type == 'r'].copy()
catalog_surveys, catalog_features = make_report_objects(river_catalogue)

river_land_use_catalog = catalog_features.df_cat.copy()
river_land_use_catalog['feature_type'] = 'r'


args = {
    'likelihood': params_l,
    'prior' : params_p,
    'data' : data_of_interest,
    'land-use-inventory' :river_land_use_catalog,
}

river_results = reports_and_forecast(args['likelihood' ], args['prior'], ldata=args['data'])
standard_river = make_standard_report(river_results, args)

combined_report = combined_results['this_report']
combined_prior_report = combined_results['prior_report']
combined_land_use = combined_results['this_land_use']


lake_report = lake_results['this_report']
lake_prior_report = lake_results['prior_report']
lake_land_use = lake_results['this_land_use']

river_report = river_results['this_report']
river_prior_report = river_results['prior_report']
river_land_use = river_results['this_land_use']

In [10]:
page_title = Markdown(f'(canton-{canton.lower()})=\n# Canton {canton}')
glue('page-title', page_title, display=False)


# section one vital statistics
section_one = Markdown(f'({canton.lower()}-vital-statistics)=\n## Vital statistics')
glue('section-one-title', section_one, display=False)


# Tab two
# feature inventroy
lake_feature_inventory = Markdown(lake_combined.feature_inventory().to_markdown())
glue('lake-combined-features', lake_feature_inventory, display=False)

# administrative boundaries
lake_administrative_boundaries = Markdown(lake_combined.administrative_boundaries().to_markdown())
glue('lake-combined-administrative-boundaries', lake_administrative_boundaries, display=False)

# material report
lake_material_report = Markdown(lake_combined.material_report.T.to_markdown())
glue('lake-combined-material-report', lake_material_report, display=False)

# histogram of prior and likelihood
lake_caption_history = Markdown(f"Survey total pcs/m {standard_lake['likelihood-labels']} versus {standard_lake['prior-labels']}. All locations considered.")
lake_histogram_of_results = histograms_standard(standard_lake['observed-values'])
glue('lake-histogram-of-results', lake_histogram_of_results, display=False)
glue('lake-caption-history', lake_caption_history, display=False)

# summary of sample results
lake_combined_summary = Markdown(apply_table_styles(lake_combined.sampling_results_summary, format_kwargs=format_kwargs).data.to_markdown())
start, end = lake_combined.date_range['start'][:4], lake_combined.date_range['end'][:4]
glue('combined-summary-label', Markdown(f'{start} - {end}'), display=False)
glue('lake-combined-summary', lake_combined_summary, display=False)

lake_likelihood_summary = Markdown(apply_table_styles(lake_report.sampling_results_summary, format_kwargs=format_kwargs).data.to_markdown())
start, end = lake_report.date_range['start'].year, lake_report.date_range['end'].year
glue('lake-summary-label', Markdown(f'{start} - {end}'), display=False)
glue('lake-likelihood-summary', lake_likelihood_summary, display=False)

lake_prior_summary = Markdown(apply_table_styles(lake_prior_report.sampling_results_summary,format_kwargs=format_kwargs).data.to_markdown())
start, end = lake_prior_report.date_range['start'].year, lake_prior_report.date_range['end'].year
glue('lake-prior-summary-label', Markdown(f'{start} - {end}'), display=False)
glue('lake-prior-summary', lake_prior_summary, display=False)

# scatterplot of results
fig, title = scatter_plot_display(combined_results, standard_combined)
glue('scatter-prior-likelihood', fig,display=False)

glue('scatter-plot-title', Markdown(title), display=False)

# most common section two
section_two = Markdown(f'({canton.lower()}-most-common)=\n## {canton} most common objects {standard_combined["likelihood-labels"]}')
glue('section-two-title', section_two, display = False)

columns = ['code', 'quantity', 'pcs/m', '% of total', 'rate']

river_most_common =  Markdown(display_selected_columns(standard_river['most-common-objects'], columns=columns, index_column='code').to_markdown())
river_ratio_most_common = standard_river['proportion-most-common']
lake_most_common =  Markdown(display_selected_columns(standard_lake['most-common-objects'], columns=columns, index_column='code').to_markdown())
lake_ratio_most_common = standard_lake['proportion-most-common']
combined_most_common = Markdown(display_selected_columns(standard_combined['most-common-objects'], columns=columns, index_column='code').to_markdown())
combined_ratio_most_common = standard_combined['proportion-most-common']

glue('combined-ratio-most-common', combined_ratio_most_common, display=False)
glue('combined-most-common', combined_most_common, display=False)

glue('river-ratio-most-common', river_ratio_most_common, display=False)
glue('river-most-common', river_most_common, display=False)

glue('lake-ratio-most-common', lake_ratio_most_common, display=False)
glue('lake-most-common', lake_most_common, display=False)


# section three land use
section_three = Markdown(f'({canton.lower()}-landuse)=\n## {canton} land cover and land use {standard_combined["likelihood-labels"]}')
glue('section-three-title', section_three, display = False)

combined_profile = Markdown((combined_land_use.n_samples_per_feature()/combined_report.number_of_samples).to_markdown())
lake_profile = Markdown((lake_land_use.n_samples_per_feature()/lake_report.number_of_samples).to_markdown())
river_profile = Markdown((river_land_use.n_samples_per_feature()/river_report.number_of_samples).to_markdown())

glue('combined-profile', combined_profile, display=False)
glue('lake-profile', lake_profile, display=False)
glue('river-profile', river_profile, display=False)

# section three rate per feature

combined_rate = Markdown(combined_land_use.rate_per_feature().to_markdown())
lake_rate = Markdown(lake_land_use.rate_per_feature().to_markdown())
river_rate = Markdown(river_land_use.rate_per_feature().to_markdown())

glue('combined-rate', combined_rate, display=False)
glue('lake-rate', lake_rate, display=False)
glue('river-rate', river_rate, display=False)

# section four
# combined forecasts
combined_forecasted_distributions = ecdf_plots_standard(standard_combined['forecasted-values'])
glue('forecasted-cumulative-dist', combined_forecasted_distributions, display=False)

combined_forecast_99 = Markdown(apply_table_styles(standard_combined['observed-99-forecast'], format_kwargs=format_kwargs).data.to_markdown())
combined_forecast_max = Markdown(apply_table_styles(standard_combined['observed-max-forecast'], format_kwargs=format_kwargs).data.to_markdown())
combined_forecast_weighted = Markdown(apply_table_styles(standard_combined['weighted-forecast'], format_kwargs=format_kwargs).data.to_markdown())

glue('combined-99-forecast', combined_forecast_99, display=False)
glue('combined-max-forecast', combined_forecast_max, display=False)
glue('combined-weighted-forecast', combined_forecast_weighted, display=False)

# lake forecast
lake_forecasted_distributions = ecdf_plots_standard(standard_lake['forecasted-values'])
glue('lake-cumulative-dist', lake_forecasted_distributions, display=False)

lake_forecast_99 = Markdown(apply_table_styles(standard_lake['observed-99-forecast'], format_kwargs=format_kwargs).data.to_markdown())
lake_forecast_max = Markdown(apply_table_styles(standard_lake['observed-max-forecast'], format_kwargs=format_kwargs).data.to_markdown())
lake_forecast_weighted = Markdown(apply_table_styles(standard_lake['weighted-forecast'], format_kwargs=format_kwargs).data.to_markdown())

glue('lake-99-forecast', lake_forecast_99, display=False)
glue('lake-max-forecast', lake_forecast_max, display=False)
glue('lake-weighted-forecast', lake_forecast_weighted, display=False)


# river forecast
river_forecasted_distributions = ecdf_plots_standard(standard_river['forecasted-values'])
glue('river-cumulative-dist', river_forecasted_distributions, display=False)

river_forecast_99 = Markdown(apply_table_styles(standard_river['observed-99-forecast'], format_kwargs=format_kwargs).data.to_markdown())
river_forecast_max = Markdown(apply_table_styles(standard_river['observed-max-forecast'], format_kwargs=format_kwargs).data.to_markdown())
river_forecast_weighted = Markdown(apply_table_styles(standard_river['weighted-forecast'], format_kwargs=format_kwargs).data.to_markdown())

glue('river-99-forecast', river_forecast_99, display=False)
glue('river-max-forecast', river_forecast_max, display=False)
glue('river-weighted-forecast', river_forecast_weighted, display=False)

# section five - summary by lake and river

summardata = all_data_of_interest.groupby(['sample_id', 'feature_type','feature_name'], as_index=False).agg({'pcs/m':'sum', 'quantity':'sum'})
# lakes
feature_individual_summary = summardata.groupby(['feature_type','feature_name'], as_index=False).agg({'sample_id':'nunique', 'pcs/m':'mean', 'quantity':'sum'})

lakes_individual_summary = feature_individual_summary[feature_individual_summary.feature_type == 'l'][['feature_name', 'sample_id', 'pcs/m']].set_index('feature_name', drop=True)
rivers_individual_summary = feature_individual_summary[feature_individual_summary.feature_type == 'r'][['feature_name', 'sample_id', 'pcs/m']].set_index('feature_name', drop=True)

glue('lakes-i-summary', Markdown(lakes_individual_summary.to_markdown()), display=False)
glue('rivers-i-summary', Markdown(rivers_individual_summary.to_markdown()), display=False)

cols = ['feature_type','city', 'sample_id', 'quantity', 'pcs/m']

dxl = all_data_combined.df[cols].groupby(['feature_type', 'city', 'sample_id'], as_index=False).agg({'pcs/m':'sum', 'quantity':'sum'})
dxl = dxl.groupby(['feature_type', 'city'], as_index=False).agg({'sample_id':'nunique', 'pcs/m':'mean', 'quantity':'sum'})
glue('municipal-lakes', Markdown(dxl[dxl.feature_type == 'l'][['city', 'sample_id', 'quantity', 'pcs/m']].set_index('city', drop=True).to_markdown()), display=False)
glue('municipal-river', Markdown(dxl[dxl.feature_type == 'r'][['city', 'sample_id', 'quantity', 'pcs/m']].set_index('city', drop=True).to_markdown()), display=False)

```{glue:md} page-title
:format: myst
```
__Density of trash along lakes and rivers__

This is a sample cantonal report. The structure and the format are based off of the federal report, ([IQAASL](https://hammerdirt-analyst.github.io/IQAASL-End-0f-Sampling-2021/)). This version is intended for use as a decsion support tool. Thus, the user is expected to be familiar with the results in the federal report and the methods described in the _Guide for Monitoring Marine Litter on European Seas_ 
([The guide](https://mcc.jrc.ec.europa.eu/main/dev.py?N=41&O=439&titre_chap=TG%20Litter&titre_page=Guidance%20for%20the%20Monitoring%20of%20Marine%20Litter)).

:::{dropdown} Initial assessments: stakeholder discussion and priorities.

Stakeholders should consider the following questions while consulting the report:

__Sampling distribution:__

1. Are the major rivers and lakes included?
2. __Does the sampling distribution reflect the topography and land-use of the canton?__
3.  __Are the municipalities of strategic importance to the canton included?__
4. __Are their locations in the canton that should have been surveyed, according to cantonal priorities?__

__Other sources of information:__

1. __How do these results compare to assessments from other sources of information ?__
   * EAWAG, EMPA other university or govt institutions
   * NGOs or volunteer assessments
   * Municipal and cantonal audits
   * Are these information sources comparable ?
   * What other sources of information are available ?
2. Combined with t
   * In your opinion how can these different sources be 

__Survey results:__

1. Was their more or less observed in 2021 vs the prior results?
   * What do the forecasts indicate?
   * Does this align with your personal experience?

__Objects of interest:__

1. Are the _most common_ currently the focus of reduction or prevention campaigns?
   * __How does the canton decide priorties in this regard?__
   * __Did or does the object appear in any regional action plan or strategy?__
2. __For objects that are the focus of prevention or reduction campaigns:__
   * Are they on the _most common objects_ list now?
   * Do they appear in the inventory ? What % of the total are they? How often are they found ?
   * What excatly was the mechanism or process that was intended to reduce the presence of the object?
3. Are their products of regional interest that should be included in the cantonal report?

__With respect to the amount of resources attributed to prevention and mitigation:__ 

1. Are these results in line with expectations?
2. How do you feel the current strategy will help reduce the observed quantities?

:::

In [11]:
# Create a GeoDataFrame from the list of locations
dbc = gpd.read_file('data/ignorethis/shapes/kantons.shp')
dbc = dbc.to_crs(epsg=4326)
dbc = dbc[dbc.NAME == canton].copy()
dbckey = dbc[['NAME', 'KANTONSNUM']].set_index('NAME')
dbckey = dbckey.drop_duplicates()
thiscanton = dbckey.loc[canton, 'KANTONSNUM']
db = gpd.read_file('data/ignorethis/shapes/municipalities.shp')
db = db.to_crs(epsg=4326)
thesecities = db[db.KANTONSNUM == thiscanton]
surveyedcities = all_data_of_interest.city.unique()

bounds = dbc.total_bounds
minx, miny, maxx, maxy = bounds


rivers = gpd.read_file('data/ignorethis/shapes/rivers.shp')
rivers = rivers.to_crs(epsg=4326)
# Filter the background layer to cover the bounding box
rivers_within_bounds = rivers.clip(dbc, keep_geom_type=True) # cx[minx:maxx, miny:maxy]


lakes = gpd.read_file('data/ignorethis/shapes/lakes.shp')
lakes = lakes.to_crs(epsg=4326)
lakes_within_bounds = lakes.clip(dbc, keep_geom_type=True) # [minx:maxx, miny:maxy]

# Define the plot
fig, ax = plt.subplots(figsize=(12,10))

citymap = thesecities.plot(ax=ax, edgecolor='black', facecolor='None', linewidth=.1)

surveyed = thesecities[thesecities.NAME.isin(surveyedcities)].plot(ax=ax, color='salmon', alpha=0.6)

dbc.plot(ax=ax, edgecolor='black', facecolor='None', linewidth=.4)
rivers_within_bounds.plot(ax=ax, edgecolor='dodgerblue', alpha=.4)
lakes_within_bounds.plot(ax=ax, edgecolor='dodgerblue', color='dodgerblue', linewidth=.2, alpha=.4)

lakes = lake_combined.sample_results
rivers = river_combined.sample_results

lakes_locations,  map_boundsa = map_markers(lakes)
lakes_locations.plot(ax=ax, color='grey', markersize=80, zorder=5)

rivers_locations,  map_boundsb = map_markers(rivers)
rivers_locations.plot(ax=ax, color='blue', markersize=80, zorder=5)

ax.set_title(f'Survey locations {canton}')
plt.xlabel('')
plt.ylabel('')

plt.axis('off')

legend_elements = [
    Line2D([0], [0], marker='o', color='w', label='lakes', markersize=10, markeredgecolor='w', markerfacecolor='grey'),
    Line2D([0], [0], marker='o', color='w', label='rivers', markersize=10, markeredgecolor='w', markerfacecolor='blue')
]

plt.legend(handles=legend_elements, loc='upper right')

glue('map-of-survey-locations', fig, display=False)
plt.close()

:::::{dropdown} Map of survey locations



::::{grid}
:padding: 2

:::{grid-item-card}

```{glue} map-of-survey-locations
```

:::
::::
:::::

```{glue:md} section-one-title
:format: myst
```
Use case: [Is their more or less ?](more-or-less)

Ref : [Survey report class](surveyreporter)


````````{grid} 1 2 2 2
:gutter: 1


```````{grid-item-card}
:columns: 12 4 4 4

```{glue:md} combined-summary-label
:format: myst
```
^^^ 

__Features__
```{glue:md} lake-combined-features
:format: myst
```

__Administration__
```{glue:md} lake-combined-administrative-boundaries
:format: myst
```
```````

```````{grid-item-card}
:columns: 12 8 8 8
:shadow: none


```{glue} lake-histogram-of-results
```

+++
```{glue:md} lake-caption-history
:format: myst
```
```````

```````{grid-item}
:columns: 12
__Material composition__

```{glue:md} lake-combined-material-report
:format: myst
```
```````

````````

````````{grid} 1 3 3 3
:padding: 1

```````{grid-item-card}
:shadow: none
:padding: 0

```{glue:md} combined-summary-label
:format: myst
```
^^^
```{glue:md} lake-combined-summary
:format: myst
```
```````

```````{grid-item-card}
:shadow: none
:padding: 0

```{glue:md} lake-summary-label
:format: myst
```
^^^

```{glue:md} lake-likelihood-summary
:format: myst
```
```````

```````{grid-item-card}
:shadow: none
:padding: 0
```{glue:md} lake-prior-summary-label
:format: myst
```
^^^

```{glue:md} lake-prior-summary
:format: myst
```
```````

````````

:::::{dropdown}  How did we get this data ?



::::{grid}
:padding: 2

:::{grid-item-card}

```{glue:md} scatter-plot-title
:format: myst
```
^^^ 

```{glue} scatter-prior-likelihood
```
+++
The data is a combination of observations from variety of groups in Switerland since 2015. The observations were recorded using an interpretation of the _Guide for Monitoring Marine Litter on European Seas_ [The guide](https://mcc.jrc.ec.europa.eu/main/dev.py?N=41&O=439&titre_chap=TG%20Litter&titre_page=Guidance%20for%20the%20Monitoring%20of%20Marine%20Litter). The guide and the monitoring of beach litter are part of decades of research, here is the brief history [A Brief History of Marine Litter Research](https://link.springer.com/chapter/10.1007/978-3-319-16510-3_1).
:::
::::

__Common sense guidance:__

1. The data should be considered as a reasonable estimate of the minimum amount of trash on the ground at the time of the survey.
2. There are many sources of variance. We have considered the following:
   * litter density between sampling groups.
   * litter density with respect to topographical features.
3. There are differences in detect-ability and appearance for items of the same classification that are due to the effects of decomposition.
4. Many surveyors are volunteers and have different levels of experience or physical constraints that limit what will actually be collected and counted.
:::::

:::{dropdown} How to make a report

__Survey and Land use__

A report is the implementation of a `SurveyReport` and a `LandUseReport`. The `SurveyReport` is the basic 
element and does the initial aggregating and descriptive statistics for a query.

The land-use-report accepts `SurveyReport.sample_results` and assigns the land-use attributes to the record. The 
land-use-report provides the baseline assessment of litter density with reference to the surrounding environment. 

The assessment accepts as variables the proportion of available space that a topographical feature occupies in a 
circle of $\pi r² \text{ where r = 1 500 meters}$ and the center of that circle is the survey location. 
These proportions are compared to the `average pieces per meter` for an object or group of objects.


__Create a report__

A report can be intiated by providing the name of the canton. If your canton does not appear this is because we have no data. The prior dates will be calculated automatically, by taking all data prior to the start date of the querry.

```{code} python

import reports
import geospatial
import gridforecast

# suppose you have defined your data into df
observed_dates = {'start':'2020-01-01', 'end':'2021-12-31'}

# everything that was seen before
prior_dates = {'start':'2015-11-15', 'end':'2019-12-31'}

# name the canton
canton = 'Bern'

# define the data of interest
data_of_interest = {'canton':canton, 'date_range':observed_dates}

# load the data
df = session_config.collect_survey_data()

# filter the data. 
filtered_data, locations = gridforeacast.filter_data(df, data_of_interest)

# make a survey report
this_report = reports.SurveyReport(dfc=filtered_data)

# generate the parameters for the landuse report
target_df = this_report.sample_results
features = geospatial.collect_topo_data(locations=target_df.location.unique())

# make a landuse report
this_land_use = geospatial.LandUseReport(target_df, features)
```

Each report and the inference method are documented: [SurveyReport](surveyreporter), [LandUseReport](landusereporter), [GridForecaster](gridforecaster)
:::


```{glue:md} section-two-title
:format: myst
```

Use case: [Interpret observed values](use-case-observed), [Is their more or less ?](more-or-less), [What % are identified](indentified-items)

Ref : [Survey report class](surveyreporter)

``````````{tab-set}

`````````{tab-item} All data

`````{grid} 1 2 2 2 
````{grid-item}
:columns: 4 4 4 4

```{glue:md} combined-ratio-most-common
:format: myst
```

The most common objects from the selected data. The most common objects are a combination of the top ten most abundant objects and those objects that are found in more than 50% of the samples. Some objects are found frequently but at low quantities.Other objects are found in fewer samples but at higher quantities.


````
````{grid-item-card}
:columns: 8 8 8 8 
:shadow: none

```{glue:md} combined-most-common
:format: myst
```
````
`````
`````````

`````````{tab-item} Lakes

`````{grid} 1 2 2 2 
````{grid-item}
:columns: 4 4 4 4

```{glue:md} lake-ratio-most-common
:format: myst
```

The most common objects from the selected data. The most common objects are a combination of the top ten most abundant objects and those objects that are found in more than 50% of the samples. Some objects are found frequently but at low quantities.Other objects are found in fewer samples but at higher quantities.


````
````{grid-item-card}
:columns: 8 8 8 8 
:shadow: none

```{glue:md} lake-most-common
:format: myst
```
````
`````
`````````

`````````{tab-item} Rivers

`````{grid} 1 2 2 2 
````{grid-item}
:columns: 4 4 4 4

```{glue:md} river-ratio-most-common
:format: myst
```

The most common objects from the selected data. The most common objects are a combination of the top ten most abundant objects and those objects that are found in more than 50% of the samples. Some objects are found frequently but at low quantities.Other objects are found in fewer samples but at higher quantities.


````
````{grid-item-card}
:columns: 8 8 8 8 
:shadow: none

```{glue:md} river-most-common
:format: myst
```
````
`````
`````````
``````````


```{glue:md} section-three-title
:format: myst
```
Use case: [Find the cleanest locations](cleanest-beach)

ref: [Land use report](landusereporter), [Calculating land use](calculating-land-use)

Land use refers to the measurable topographic features within a cirlce of r = 1 500 m and area = $\pi r²$ with the survey location in the middle (the buffer). The features, measured in meters squared, are given as a ratio $\frac{\text{area of feature}}{\text{area of circle}}$. Thus a location with high percentage of buildings will have a rating or value between 60% and 100%. The pcs/m rating is the average of all locations with a land-use profile of the same rating.

The land use is further divided in to two groups: cover and use. Cover refers to those topographical features that do not overlap. That is cover features are mutually exclusive, a given area of the buffer is either one or the other of the cover features. The cover features are:

* Buildings, orchards, forest, undefined, vineyards

Use refers to the activities or features that are present in one of the cover features. For example public services can be located within buildings (hospitals, schools) or in a forest (parks, nature areas). The use features are:

* Pubilc services, streets

__Streets__

The streets are measured as the length of the road network in the cirlce with r= 1 500 m and area $\pi r²$ and the survey location in the middle. The lengths for each location are normalized from 0 - 1. Thus in the table below, the locations that have the shortest road net work will be in category 1, the those with a more dense network will be higher.
<br></br>

__Land use/cover:__ distribution of samples and pcs/m by % of buffer attributed to the cover or use

`````{tab-set}

````{tab-item} All lakes and rivers
:selected: True

__% of buffer attributed to land use or cover__

```{glue:md} combined-profile
:format: myst
```
<br />

__Pieces per meter by % of buffer attributed to land use or cover__

```{glue:md} combined-rate
:format: myst
``` 

````

````{tab-item} Lakes

__% of buffer attributed to land use or cover__

```{glue:md} lake-profile
:format: myst
```

<br />

__Pieces per meter by % of buffer attributed to land use or cover__

```{glue:md} lake-rate
:format: myst
```

```` 

````{tab-item} Rivers

__% of buffer attributed to land use or cover__

```{glue:md} river-profile
:format: myst
```

<br />

__Pieces per meter by % of buffer attributed to land use or cover__

```{glue:md} lake-rate
:format: myst
```
````
`````

(bern-forecasts)=
## Forecast

Use case: [Interpret forecasted values](use-case-forecast)

ref: [Grid forecasts](forecast-class), [Empirical Bayes](gridforecaster)

The applied method would best be classified as Empirical Bayes, in the sense that the prior is derived from the data ([Bayesian Filtering and Smoothing](https://users.aalto.fi/~ssarkka/pub/cup_book_online_20131111.pdf) or [Empirical Bayes methods in classical and Bayesian inference](https://hannig.cloudapps.unc.edu/STOR757Bayes/handouts/PetroneEtAl2014.pdf)).

__Model assumptions__

1. Locations with similar land use attributes will have similar litter density rates
2. The data is a best estimate of what was present on the day of the survey
3. There are regional differences with respect to the density of specific objects
4. The locations surveyed are maintained by a public administration

``````````{tab-set}

`````````{tab-item} All data

`````{grid} 1 1 2 2

````{grid-item-card}
:columns: 12 5 5 5 

__Weighted prior:__ The weighted prior is a random sample of survey results from locations outside the report area of interest with similar land use configurations. This can be interpreted as the forecast based on the assumption that the current area of interest is similar to anyother area of country.  They are


````

````{grid-item-card}
:columns: 12 7 7 7 
:shadow: none
```{glue} forecasted-cumulative-dist
```
+++
Cumulative distribution of observed, sampling history and forecasts using to different priors.
````
`````

`````{grid} 1 3 3 3

````{grid-item}

__Given the 99% interval of the likelihood__

```{glue:md} combined-99-forecast
:format: myst
``` 

````

````{grid-item}

__Given the entire interval of the likelihood__

```{glue:md} combined-max-forecast
:format: myst
```

````

````{grid-item}

__Given the results from similar locations (weighted prior)__

```{glue:md} combined-weighted-forecast
:format: myst

```

````

`````
`````````

`````````{tab-item} Lakes

`````{grid} 1 1 2 2

````{grid-item-card}
:columns: 12 5 5 5 

__Weighted prior:__ The weighted prior is a random sample of survey results from locations outside the report area of interest with similar land use configurations. This can be interpreted as the forecast based on the assumption that the current area of interest is similar to anyother area of country.  They are


````

````{grid-item-card}
:columns: 12 7 7 7 
:shadow: none
```{glue} lake-cumulative-dist
```
+++
Cumulative distribution of observed, sampling history and forecasts using to different priors.
````
`````

`````{grid} 1 3 3 3

````{grid-item}

__Given the 99% interval of the likelihood__

```{glue:md} lake-99-forecast
:format: myst
``` 

````

````{grid-item}

__Given the entire interval of the likelihood__

```{glue:md} lake-max-forecast
:format: myst
```

````

````{grid-item}

__Given the results from similar locations (weighted prior)__

```{glue:md} lake-weighted-forecast
:format: myst

```

````

`````
`````````


`````````{tab-item} rivers

`````{grid} 1 1 2 2

````{grid-item-card}
:columns: 12 5 5 5 

__Weighted prior:__ The weighted prior is a random sample of survey results from locations outside the report area with similar land use configurations. This can be interpreted as the forecast based on the assumption that the current area of interest is similar to any other area of country.




````

````{grid-item-card}
:columns: 12 7 7 7 
:shadow: none
```{glue} river-cumulative-dist
```
+++
Cumulative distribution of observed, sampling history and forecasts using to different priors.
````
`````

`````{grid} 1 3 3 3

````{grid-item}

__Given the 99% interval of the likelihood__

```{glue:md} river-99-forecast
:format: myst
``` 

````

````{grid-item}

__Given the entire interval of the likelihood__

```{glue:md} river-max-forecast
:format: myst
```

````

````{grid-item}

__Given the results from similar locations (weighted prior)__

```{glue:md} river-weighted-forecast
:format: myst

```

````

`````
`````````
``````````

## Lakes and rivers sampled - all data

`````{grid} 2 2 2 2

````{grid-item}
**Lakes sampled**

```{glue:md} lakes-i-summary
:format: myst
```

````

````{grid-item}
**Rivers sampled**

```{glue:md} rivers-i-summary
:format: myst
```
````
`````


## Municipal results lakes and rivers

`````{grid} 2 2 2 2

````{grid-item}
**Lakes sampled**

```{glue:md} municipal-lakes
:format: myst
```

````

````{grid-item}
**Rivers sampled**

```{glue:md} municipal-river
:format: myst
```
````
`````